# Model Pretraining

#### Healthcare case studies

In [1]:
# importing the dependencies 

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

/Users/rajendran/Desktop/LLM/llm_workshop/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [3]:
# Function to generate answer for a patient question
def generate_answer(question):
    # Encode the question
    input_ids = tokenizer.encode(question, return_tensors="pt")

    # Generate answer using GPT-2 model
    with torch.no_grad():
        output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)
    
    # Decode the generated answer
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return answer

In [4]:
# Example usage
question = "What are the symptoms of COVID-19?"
answer = generate_answer(question)
print("Answer:", answer)

/Users/rajendran/Desktop/LLM/llm_workshop/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: What are the symptoms of COVID-19?

COVID-19 is a common cold. It is caused by a virus that causes a cold. It is usually caused by a virus that causes a cold. It is usually caused by a virus that causes a cold. It is usually caused by a virus that causes a cold. It is usually caused by a virus that causes a cold. It is usually caused by a virus that causes a cold. It is usually caused by a virus that


## RAG

In [5]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

In [22]:
# Sample patient question-answer pairs
patient_data = [
    {
        "question": "What are the symptoms of COVID-19?", 
        "answer": "Common symptoms of COVID-19 include fever, cough, and shortness of breath."
    },

    {
         "question": "How is diabetes diagnosed?", 
         "answer": "Diabetes is diagnosed through tests such as fasting blood sugar, oral glucose tolerance test, etc."
    },
    
    {
        "question": "What treatments are available for asthma?",
        "answer": "Treatments for asthma include bronchodilators, inhaled corticosteroids, etc."
    },
    
    {
        "question": "What is the function of the pancreas?",
        "answer": "The pancreas produces enzymes that aid in digestion and hormones such as insulin and glucagon that regulate blood sugar levels."
    },
    {
        "question": "How does chemotherapy work?",
        "answer": "Chemotherapy works by targeting and killing rapidly dividing cells, which includes cancer cells. It can be administered orally or intravenously."
    },
    {
        "question": "What is the difference between type 1 and type 2 diabetes?",
        "answer": "Type 1 diabetes is an autoimmune condition where the body does not produce insulin, while type 2 diabetes is characterized by insulin resistance or insufficient insulin production."
    },
    {
        "question": "What are the risk factors for heart disease?",
        "answer": "Risk factors for heart disease include high blood pressure, high cholesterol, smoking, obesity, diabetes, and a sedentary lifestyle."
    },
    {
        "question": "What are the symptoms of a stroke?",
        "answer": "Symptoms of a stroke include sudden numbness or weakness in the face, arm, or leg, confusion, trouble speaking or understanding, and difficulty walking."
    },
    {
        "question": "How does the immune system work?",
        "answer": "The immune system defends the body against pathogens such as bacteria, viruses, and parasites. It includes various cells, tissues, and organs that work together to identify and destroy harmful invaders."
    },
    {
        "question": "What are the benefits of regular exercise?",
        "answer": "Regular exercise can improve cardiovascular health, strengthen muscles and bones, boost mood, reduce stress, and help maintain a healthy weight."
    },
    {
        "question": "What is the recommended daily intake of water?",
        "answer": "The recommended daily intake of water varies depending on factors such as age, sex, and activity level, but a general guideline is around 8 glasses or 2 liters per day."
    },
    {
        "question": "How does the flu vaccine work?",
        "answer": "The flu vaccine works by exposing the immune system to weakened or inactive flu viruses, triggering an immune response that protects against future infections."
    },
    {
        "question": "What are the benefits of a balanced diet?",
        "answer": "A balanced diet provides essential nutrients such as vitamins, minerals, protein, carbohydrates, and fats, which are necessary for overall health, energy, and disease prevention."
    },
    {
        "question": "What is the role of antioxidants in the body?",
        "answer": "Antioxidants help neutralize harmful free radicals in the body, reducing oxidative stress and lowering the risk of chronic diseases such as cancer and heart disease."
    },
    {
        "question": "How does insulin regulate blood sugar levels?",
        "answer": "Insulin facilitates the uptake of glucose by cells, reducing blood sugar levels. It also signals the liver to store excess glucose as glycogen."
    },
    {
        "question": "What are the benefits of quitting smoking?",
        "answer": "Quitting smoking can improve lung function, reduce the risk of heart disease, stroke, and cancer, increase life expectancy, and improve overall quality of life."
    },
    {
        "question": "What are the warning signs of skin cancer?",
        "answer": "Warning signs of skin cancer include changes in the size, shape, or color of moles or other skin lesions, itching, bleeding, and the appearance of new growths."
    },
    {
        "question": "How does stress affect health?",
        "answer": "Chronic stress can negatively impact physical and mental health, leading to conditions such as high blood pressure, heart disease, depression, and anxiety."
    },
    {
      "question":"What are the symptoms of small pox?",
      "answer":"It shows a severe enlargement in parotid gland"
    }

     # Add more patient questions and corresponding answers

]

   


In [23]:
# Load pre-trained Sentence Transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
df = pd.DataFrame(patient_data)

In [24]:
# Embed the answers
answer_embeddings = model.encode(df['question'], convert_to_tensor=True)

In [25]:
def calculate_similarity(query, embeddings):
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    return cos_scores

In [26]:
def input_text_model(input_text):
    input_sentence_processed = ''.join([i for i in input_text if not i.isdigit()])
    query = input_sentence_processed
    query_similarity = calculate_similarity(query, answer_embeddings)
    top_n = 1
    top_indices = query_similarity.argsort().cpu().numpy()[-top_n:]
    similar_answers = df.loc[top_indices, 'answer'].tolist()
    return similar_answers[-1]


In [28]:
# Example usage
patient_question = "symptoms of smallpox?"
answer = input_text_model(patient_question)
print("Patient Question:", patient_question)
print("Generated Answer:", answer)

Patient Question: symptoms of smallpox?
Generated Answer: It shows a severe enlargement in parotid gland
